In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect('Chinook.db')

In [3]:
cur = conn.cursor()

In [11]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
all_tables = cursor.fetchall()

In [15]:
template = "The '{table_name}' table has columns: {columns}."

In [18]:
tables = []
for table in all_tables:
    table_name = table[0]
    columns = []
    for _, col_name, col_type, _, _, _ in conn.execute(f"pragma table_info('{table_name}')").fetchall():
        columns.append(f"{col_name} ({col_type})")
    tables.append(template.format(table_name=table_name, columns=", ".join(columns)))

In [20]:
template = """Instruction:
Given an input question, respond with syntactically correct SQLite. Be creative but the SQL must be correct. Only use the following tables:

{table_info}

Input: {{input}}
""".format(table_info="\n".join(tables))

In [33]:
tables

["The 'Album' table has columns: AlbumId (INTEGER), Title (NVARCHAR(160)), ArtistId (INTEGER)",
 "The 'Artist' table has columns: ArtistId (INTEGER), Name (NVARCHAR(120))",
 "The 'Customer' table has columns: CustomerId (INTEGER), FirstName (NVARCHAR(40)), LastName (NVARCHAR(20)), Company (NVARCHAR(80)), Address (NVARCHAR(70)), City (NVARCHAR(40)), State (NVARCHAR(40)), Country (NVARCHAR(40)), PostalCode (NVARCHAR(10)), Phone (NVARCHAR(24)), Fax (NVARCHAR(24)), Email (NVARCHAR(60)), SupportRepId (INTEGER)",
 "The 'Employee' table has columns: EmployeeId (INTEGER), LastName (NVARCHAR(20)), FirstName (NVARCHAR(20)), Title (NVARCHAR(30)), ReportsTo (INTEGER), BirthDate (DATETIME), HireDate (DATETIME), Address (NVARCHAR(70)), City (NVARCHAR(40)), State (NVARCHAR(40)), Country (NVARCHAR(40)), PostalCode (NVARCHAR(10)), Phone (NVARCHAR(24)), Fax (NVARCHAR(24)), Email (NVARCHAR(60))",
 "The 'Genre' table has columns: GenreId (INTEGER), Name (NVARCHAR(120))",
 "The 'Invoice' table has columns:

In [22]:
from langchain import OpenAI

In [31]:
llm = OpenAI(temperature=0)

In [35]:
sql_cmd = llm(template.format(input="What is Eduardo Martins's phone number?"))
sql_cmd

"\nSELECT Phone FROM Customer WHERE FirstName = 'Eduardo' AND LastName = 'Martins'"

In [28]:
cursor = conn.cursor()
cursor.execute("select * from Customer")
res = cursor.fetchall()

In [29]:
res

[(1,
  'Luís',
  'Gonçalves',
  'Embraer - Empresa Brasileira de Aeronáutica S.A.',
  'Av. Brigadeiro Faria Lima, 2170',
  'São José dos Campos',
  'SP',
  'Brazil',
  '12227-000',
  '+55 (12) 3923-5555',
  '+55 (12) 3923-5566',
  'luisg@embraer.com.br',
  3),
 (2,
  'Leonie',
  'Köhler',
  None,
  'Theodor-Heuss-Straße 34',
  'Stuttgart',
  None,
  'Germany',
  '70174',
  '+49 0711 2842222',
  None,
  'leonekohler@surfeu.de',
  5),
 (3,
  'François',
  'Tremblay',
  None,
  '1498 rue Bélanger',
  'Montréal',
  'QC',
  'Canada',
  'H2G 1A7',
  '+1 (514) 721-4711',
  None,
  'ftremblay@gmail.com',
  3),
 (4,
  'Bjørn',
  'Hansen',
  None,
  'Ullevålsveien 14',
  'Oslo',
  None,
  'Norway',
  '0171',
  '+47 22 44 22 22',
  None,
  'bjorn.hansen@yahoo.no',
  4),
 (5,
  'František',
  'Wichterlová',
  'JetBrains s.r.o.',
  'Klanova 9/506',
  'Prague',
  None,
  'Czech Republic',
  '14700',
  '+420 2 4172 5555',
  '+420 2 4172 5555',
  'frantisekw@jetbrains.com',
  4),
 (6,
  'Helena',
  'H

In [1]:
from langchain.databases.sqlite import SQLiteDatabase
from langchain.chains.database.base import DatabaseChain
from langchain import OpenAI

In [3]:
db = SQLiteDatabase("../notebooks/Chinook.db")
llm = OpenAI(temperature=0)
db_chain = DatabaseChain(llm=llm, database=db)

In [4]:
db_chain.query("What is Eduardo Martins's phone number?")



SELECT Phone FROM Customer WHERE FirstName = 'Eduardo' AND LastName = 'Martins'


"[('+55 (11) 3033-5446',)]"

In [7]:
db.run("select * from Album")

'[(1, \'For Those About To Rock We Salute You\', 1), (2, \'Balls to the Wall\', 2), (3, \'Restless and Wild\', 2), (4, \'Let There Be Rock\', 1), (5, \'Big Ones\', 3), (6, \'Jagged Little Pill\', 4), (7, \'Facelift\', 5), (8, \'Warner 25 Anos\', 6), (9, \'Plays Metallica By Four Cellos\', 7), (10, \'Audioslave\', 8), (11, \'Out Of Exile\', 8), (12, \'BackBeat Soundtrack\', 9), (13, \'The Best Of Billy Cobham\', 10), (14, \'Alcohol Fueled Brewtality Live! [Disc 1]\', 11), (15, \'Alcohol Fueled Brewtality Live! [Disc 2]\', 11), (16, \'Black Sabbath\', 12), (17, \'Black Sabbath Vol. 4 (Remaster)\', 12), (18, \'Body Count\', 13), (19, \'Chemical Wedding\', 14), (20, \'The Best Of Buddy Guy - The Millenium Collection\', 15), (21, \'Prenda Minha\', 16), (22, \'Sozinho Remix Ao Vivo\', 16), (23, \'Minha Historia\', 17), (24, \'Afrociberdelia\', 18), (25, \'Da Lama Ao Caos\', 18), (26, \'Acústico MTV [Live]\', 19), (27, \'Cidade Negra - Hits\', 19), (28, \'Na Pista\', 20), (29, \'Axé Bahia 200

In [6]:
!pip install sqlalchemy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 5.6 MB/s eta 0:00:00m eta 0:00:010:00:01

[notice] A new release of pip available: 22.2.2 -> 22.3
[notice] To update, run: pip install --upgrade pip


In [15]:
from sqlalchemy import create_engine
from sqlalchemy import inspect
inspector = inspect(engine)

for table_name in inspector.get_table_names():
    for column in inspector.get_columns(table_name):
        print("Column: %s" % column['name'] + str(column["type"]))

Column: AlbumIdINTEGER
Column: TitleNVARCHAR(160)
Column: ArtistIdINTEGER
Column: ArtistIdINTEGER
Column: NameNVARCHAR(120)
Column: CustomerIdINTEGER
Column: FirstNameNVARCHAR(40)
Column: LastNameNVARCHAR(20)
Column: CompanyNVARCHAR(80)
Column: AddressNVARCHAR(70)
Column: CityNVARCHAR(40)
Column: StateNVARCHAR(40)
Column: CountryNVARCHAR(40)
Column: PostalCodeNVARCHAR(10)
Column: PhoneNVARCHAR(24)
Column: FaxNVARCHAR(24)
Column: EmailNVARCHAR(60)
Column: SupportRepIdINTEGER
Column: EmployeeIdINTEGER
Column: LastNameNVARCHAR(20)
Column: FirstNameNVARCHAR(20)
Column: TitleNVARCHAR(30)
Column: ReportsToINTEGER
Column: BirthDateDATETIME
Column: HireDateDATETIME
Column: AddressNVARCHAR(70)
Column: CityNVARCHAR(40)
Column: StateNVARCHAR(40)
Column: CountryNVARCHAR(40)
Column: PostalCodeNVARCHAR(10)
Column: PhoneNVARCHAR(24)
Column: FaxNVARCHAR(24)
Column: EmailNVARCHAR(60)
Column: GenreIdINTEGER
Column: NameNVARCHAR(120)
Column: InvoiceIdINTEGER
Column: CustomerIdINTEGER
Column: InvoiceDateD

In [14]:
column

{'name': 'AlbumId',
 'type': INTEGER(),
 'nullable': False,
 'default': None,
 'autoincrement': 'auto',
 'primary_key': 1}

In [9]:
engine = create_engine("sqlite:///../notebooks/Chinook.db")

In [11]:
engine.()

/Users/harrisonchase/workplace/langchain/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  """Entry point for launching an IPython kernel.


['Album',
 'Artist',
 'Customer',
 'Employee',
 'Genre',
 'Invoice',
 'InvoiceLine',
 'MediaType',
 'Playlist',
 'PlaylistTrack',
 'Track']